In [1]:
from reader import parse_data, list_to_freq_dict, parse_data_test
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import pprint
import os
import json
import numpy as np
import multiprocessing
import time

pp = pprint.PrettyPrinter(indent=4)

In [2]:
def load_dataset(path, training=True):

    if training == True:
        word_list = []
        tag_list = []
        word_tags_list = []
        
        for subdir, dirs, files in os.walk(path):
            for file in files:

                fileName = subdir + '/' + str(file)
                words, tags, word_tags = parse_data(fileName)

                word_list.extend(words)
                tag_list.extend(tags)
                word_tags_list.extend(word_tags)
            
        return word_list, tag_list, word_tags_list
        
    else:
        all_tuples = []
        
        for subdir, dirs, files in os.walk(path):
            for file in files:

                fileName = subdir + '/' + str(file)
                tuples = parse_data_test(fileName)
                all_tuples.extend(tuples)

        return all_tuples

In [3]:
# Load Dataset

train_path = 'Train-corpus/'
test_path = 'Test-corpus/'

# word_list, tag_list, word_tags_list = load_dataset(train_path)
test_tuples = load_dataset(test_path, training=False)

In [ ]:
# Generate Frequency Dict for words

word_dict = list_to_freq_dict(word_list)

with open('words.json', 'w') as outfile:
    json.dump(word_dict, outfile, indent=4)

In [ ]:
# Print top 10 words

k1 = Counter(word_dict)
top_words = k1.most_common(10)
print('Top 10 Words are: ')
for i in top_words:
    print(i[0], " :", i[1], " ")

In [ ]:
# Plot Top 10 Words

keys, values = [i[0] for i in top_words], [i[1] for i in top_words]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(keys,values)
plt.show()

patches, texts = plt.pie(values, labels=keys)
plt.legend(patches, keys, loc="best")
plt.savefig('words_chart.png')
plt.show()

In [ ]:
# Generate Frequency Dict for tags

tag_dict = list_to_freq_dict(tag_list)
with open('tags.json', 'w') as outfile:
    json.dump(tag_dict, outfile, indent=4)

In [ ]:
# Print Top 10 Tags

k1 = Counter(tag_dict)
top_tags = k1.most_common(10)
print('Top 10 Tags are: ')
for i in top_tags:
    print(i[0], " :", i[1], " ")

In [ ]:
# Plot Top 10 Tags

keys, values = [i[0] for i in top_tags], [i[1] for i in top_tags]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(keys,values)
plt.show()

patches, texts = plt.pie(values, labels=keys)
plt.legend(patches, keys, loc="best")
plt.savefig('tags_chart.png')
plt.show()

In [ ]:
# Generate Frequency Dict for word_tags

word_tags_dict = list_to_freq_dict(word_tags_list)
with open('word_tags.json', 'w') as outfile:
    json.dump(word_tags_dict, outfile, indent=4)

In [4]:
# Load JSON Files

with open('words.json') as f:
    word_dict = json.load(f)
with open('tags.json') as f:
    tag_dict = json.load(f)
with open('word_tags.json') as f:
    word_tags_dict = json.load(f)

In [10]:
print(len(tag_dict))
print(len(word_dict))
print(len(word_tags_dict))

57
192634
252564


In [6]:
def probability_word_given_tag(word, tag):
    # (count word_tag)/(count of tag)
    
    if word+'_'+tag in word_tags_dict.keys():
        
        count_word_tag = word_tags_dict[word+'_'+tag]
        count_tag = tag_dict[tag]

        return count_word_tag/count_tag
    
    else:
        return 0

In [ ]:
def probability_tag(tag):
    count_tag = tag_dict[tag]

In [7]:
print(probability_word_given_tag('a','AT0'))

0.23395366546859694


In [13]:
def best_tag_for_word(word):
    max_prob = 0
    most_prob_tag = 'none'
    for tag in tag_dict.keys():
        p_word_tag = probability_word_given_tag(word,tag)
        p_tag = tag_dict[tag]
        
        p_tag_word = (p_word_tag*p_tag)
        
        if max_prob < p_tag_word:
            max_prob = p_tag_word
            most_prob_tag = tag
    
    return most_prob_tag

In [ ]:
def best_tag_for_word(word):
    max_count = 0
    most_prob_word_tag = 'word_none'
    for word_tag in word_tags_dict.keys():
        count = word_tags_dict[word_tag]
        if word in word_tag:
            if max_count < count:
                max_count = count
                most_prob_word_tag = word_tag
    
    return most_prob_word_tag.split('_')[1]

In [16]:
print(best_tag_for_word('of'))

PRF


In [17]:
print(len(test_tuples))

3630025


In [18]:
def check_prediction(word_tuple):
    
    word, label = word_tuple
    logit = best_tag_for_word(word)
    if logit in label:
        return True
    else:
        return False

In [22]:
def calculate(x):
    a_pool = multiprocessing.Pool()
    result = a_pool.map(check_prediction, x)

    curr_correct = sum(result)
    curr_incorrect = 1000 - curr_correct
    
    return curr_correct, curr_incorrect
    
def get_model_accuracy(test_tuples):
    
    correct = 0
    incorrect = 0
    
    print("Total: %d" % len(test_tuples))
    print('\n')
    
    for i in range(0,3620):
        
        t0 = time.process_time()
        
        x = test_tuples[i*1000: (i+1)*1000]
        
        curr_correct, curr_incorrect = calculate(x)
        
        correct = correct + curr_correct
        incorrect = incorrect + curr_incorrect
        
        print("Evaluated: %d " % ((i+1)*1000))
        print("Accuracy = %.06f " % (correct/(correct+incorrect)))
        print("Time Taken: %.2f \n " % (time.process_time()-t0))
        
    x = test_tuples[3620*1000:]
    curr_correct, curr_incorrect = calculate(x)
    correct = correct + curr_correct
    incorrect = incorrect + curr_incorrect
    
    print("Evaluated: %d " % (len(test_tuples))) 
    
    print('\n')
    print("Final Accuracy = %.06f"  % (correct/(correct+incorrect)))

In [23]:
get_model_accuracy(test_tuples)

Total: 3630025


Evaluated: 1000 
Accuracy = 0.942000 
Time Taken: 0.15 
 
Evaluated: 2000 
Accuracy = 0.944500 
Time Taken: 0.15 
 
Evaluated: 3000 
Accuracy = 0.936000 
Time Taken: 0.15 
 
Evaluated: 4000 
Accuracy = 0.936500 
Time Taken: 0.15 
 
Evaluated: 5000 
Accuracy = 0.933000 
Time Taken: 0.16 
 
Evaluated: 6000 
Accuracy = 0.933333 
Time Taken: 0.15 
 
Evaluated: 7000 
Accuracy = 0.933714 
Time Taken: 0.15 
 
Evaluated: 8000 
Accuracy = 0.932875 
Time Taken: 0.15 
 
Evaluated: 9000 
Accuracy = 0.931000 
Time Taken: 0.15 
 
Evaluated: 10000 
Accuracy = 0.931000 
Time Taken: 0.15 
 
Evaluated: 11000 
Accuracy = 0.931455 
Time Taken: 0.15 
 
Evaluated: 12000 
Accuracy = 0.931500 
Time Taken: 0.15 
 
Evaluated: 13000 
Accuracy = 0.931846 
Time Taken: 0.15 
 
Evaluated: 14000 
Accuracy = 0.931143 
Time Taken: 0.15 
 
Evaluated: 15000 
Accuracy = 0.932267 
Time Taken: 0.15 
 
Evaluated: 16000 
Accuracy = 0.933000 
Time Taken: 0.15 
 
Evaluated: 17000 
Accuracy = 0.931882 
Time Take

Evaluated: 140000 
Accuracy = 0.895707 
Time Taken: 0.15 
 
Evaluated: 141000 
Accuracy = 0.895397 
Time Taken: 0.15 
 
Evaluated: 142000 
Accuracy = 0.895261 
Time Taken: 0.15 
 
Evaluated: 143000 
Accuracy = 0.895140 
Time Taken: 0.15 
 
Evaluated: 144000 
Accuracy = 0.895090 
Time Taken: 0.15 
 
Evaluated: 145000 
Accuracy = 0.895097 
Time Taken: 0.15 
 
Evaluated: 146000 
Accuracy = 0.895301 
Time Taken: 0.15 
 
Evaluated: 147000 
Accuracy = 0.895408 
Time Taken: 0.15 
 
Evaluated: 148000 
Accuracy = 0.895338 
Time Taken: 0.15 
 
Evaluated: 149000 
Accuracy = 0.895376 
Time Taken: 0.15 
 
Evaluated: 150000 
Accuracy = 0.895360 
Time Taken: 0.15 
 
Evaluated: 151000 
Accuracy = 0.895523 
Time Taken: 0.15 
 
Evaluated: 152000 
Accuracy = 0.895566 
Time Taken: 0.15 
 
Evaluated: 153000 
Accuracy = 0.895542 
Time Taken: 0.15 
 
Evaluated: 154000 
Accuracy = 0.895442 
Time Taken: 0.15 
 
Evaluated: 155000 
Accuracy = 0.895465 
Time Taken: 0.15 
 
Evaluated: 156000 
Accuracy = 0.895417 


Evaluated: 277000 
Accuracy = 0.898480 
Time Taken: 0.16 
 
Evaluated: 278000 
Accuracy = 0.898540 
Time Taken: 0.15 
 
Evaluated: 279000 
Accuracy = 0.898645 
Time Taken: 0.15 
 
Evaluated: 280000 
Accuracy = 0.898693 
Time Taken: 0.15 
 
Evaluated: 281000 
Accuracy = 0.898733 
Time Taken: 0.15 
 
Evaluated: 282000 
Accuracy = 0.898844 
Time Taken: 0.15 
 
Evaluated: 283000 
Accuracy = 0.898982 
Time Taken: 0.16 
 
Evaluated: 284000 
Accuracy = 0.899063 
Time Taken: 0.17 
 
Evaluated: 285000 
Accuracy = 0.899109 
Time Taken: 0.17 
 
Evaluated: 286000 
Accuracy = 0.899161 
Time Taken: 0.15 
 
Evaluated: 287000 
Accuracy = 0.899279 
Time Taken: 0.15 
 
Evaluated: 288000 
Accuracy = 0.899410 
Time Taken: 0.15 
 
Evaluated: 289000 
Accuracy = 0.899561 
Time Taken: 0.15 
 
Evaluated: 290000 
Accuracy = 0.899655 
Time Taken: 0.17 
 
Evaluated: 291000 
Accuracy = 0.899732 
Time Taken: 0.16 
 
Evaluated: 292000 
Accuracy = 0.899760 
Time Taken: 0.16 
 
Evaluated: 293000 
Accuracy = 0.899846 


Evaluated: 414000 
Accuracy = 0.902068 
Time Taken: 0.17 
 
Evaluated: 415000 
Accuracy = 0.901998 
Time Taken: 0.17 
 
Evaluated: 416000 
Accuracy = 0.901918 
Time Taken: 0.19 
 
Evaluated: 417000 
Accuracy = 0.901871 
Time Taken: 0.17 
 
Evaluated: 418000 
Accuracy = 0.901864 
Time Taken: 0.17 
 
Evaluated: 419000 
Accuracy = 0.901826 
Time Taken: 0.17 
 
Evaluated: 420000 
Accuracy = 0.901824 
Time Taken: 0.17 
 
Evaluated: 421000 
Accuracy = 0.901713 
Time Taken: 0.16 
 
Evaluated: 422000 
Accuracy = 0.901645 
Time Taken: 0.17 
 
Evaluated: 423000 
Accuracy = 0.901662 
Time Taken: 0.15 
 
Evaluated: 424000 
Accuracy = 0.901719 
Time Taken: 0.15 
 
Evaluated: 425000 
Accuracy = 0.901753 
Time Taken: 0.16 
 
Evaluated: 426000 
Accuracy = 0.901730 
Time Taken: 0.17 
 
Evaluated: 427000 
Accuracy = 0.901745 
Time Taken: 0.17 
 
Evaluated: 428000 
Accuracy = 0.901757 
Time Taken: 0.15 
 
Evaluated: 429000 
Accuracy = 0.901753 
Time Taken: 0.15 
 
Evaluated: 430000 
Accuracy = 0.901698 


Evaluated: 551000 
Accuracy = 0.900789 
Time Taken: 0.15 
 
Evaluated: 552000 
Accuracy = 0.900781 
Time Taken: 0.15 
 
Evaluated: 553000 
Accuracy = 0.900776 
Time Taken: 0.15 
 
Evaluated: 554000 
Accuracy = 0.900749 
Time Taken: 0.15 
 
Evaluated: 555000 
Accuracy = 0.900706 
Time Taken: 0.15 
 
Evaluated: 556000 
Accuracy = 0.900761 
Time Taken: 0.15 
 
Evaluated: 557000 
Accuracy = 0.900829 
Time Taken: 0.15 
 
Evaluated: 558000 
Accuracy = 0.900884 
Time Taken: 0.15 
 
Evaluated: 559000 
Accuracy = 0.900962 
Time Taken: 0.15 
 
Evaluated: 560000 
Accuracy = 0.901012 
Time Taken: 0.15 
 
Evaluated: 561000 
Accuracy = 0.901061 
Time Taken: 0.15 
 
Evaluated: 562000 
Accuracy = 0.901100 
Time Taken: 0.15 
 
Evaluated: 563000 
Accuracy = 0.901165 
Time Taken: 0.15 
 
Evaluated: 564000 
Accuracy = 0.901229 
Time Taken: 0.15 
 
Evaluated: 565000 
Accuracy = 0.901269 
Time Taken: 0.14 
 
Evaluated: 566000 
Accuracy = 0.901284 
Time Taken: 0.15 
 
Evaluated: 567000 
Accuracy = 0.901296 


...

Evaluated: 2987000 
Accuracy = 0.904490 
Time Taken: 0.15 
 
Evaluated: 2988000 
Accuracy = 0.904494 
Time Taken: 0.15 
 
Evaluated: 2989000 
Accuracy = 0.904488 
Time Taken: 0.15 
 
Evaluated: 2990000 
Accuracy = 0.904486 
Time Taken: 0.15 
 
Evaluated: 2991000 
Accuracy = 0.904486 
Time Taken: 0.15 
 
Evaluated: 2992000 
Accuracy = 0.904486 
Time Taken: 0.15 
 
Evaluated: 2993000 
Accuracy = 0.904487 
Time Taken: 0.15 
 
Evaluated: 2994000 
Accuracy = 0.904495 
Time Taken: 0.15 
 
Evaluated: 2995000 
Accuracy = 0.904497 
Time Taken: 0.15 
 
Evaluated: 2996000 
Accuracy = 0.904495 
Time Taken: 0.15 
 
Evaluated: 2997000 
Accuracy = 0.904504 
Time Taken: 0.15 
 
Evaluated: 2998000 
Accuracy = 0.904503 
Time Taken: 0.15 
 
Evaluated: 2999000 
Accuracy = 0.904508 
Time Taken: 0.15 
 
Evaluated: 3000000 
Accuracy = 0.904506 
Time Taken: 0.15 
 
Evaluated: 3001000 
Accuracy = 0.904508 
Time Taken: 0.15 
 
Evaluated: 3002000 
Accuracy = 0.904511 
Time Taken: 0.15 
 
Evaluated: 3003000 
Accu

Evaluated: 3122000 
Accuracy = 0.904627 
Time Taken: 0.15 
 
Evaluated: 3123000 
Accuracy = 0.904633 
Time Taken: 0.15 
 
Evaluated: 3124000 
Accuracy = 0.904641 
Time Taken: 0.15 
 
Evaluated: 3125000 
Accuracy = 0.904651 
Time Taken: 0.15 
 
Evaluated: 3126000 
Accuracy = 0.904656 
Time Taken: 0.15 
 
Evaluated: 3127000 
Accuracy = 0.904665 
Time Taken: 0.15 
 
Evaluated: 3128000 
Accuracy = 0.904670 
Time Taken: 0.15 
 
Evaluated: 3129000 
Accuracy = 0.904679 
Time Taken: 0.15 
 
Evaluated: 3130000 
Accuracy = 0.904686 
Time Taken: 0.15 
 
Evaluated: 3131000 
Accuracy = 0.904690 
Time Taken: 0.16 
 
Evaluated: 3132000 
Accuracy = 0.904697 
Time Taken: 0.16 
 
Evaluated: 3133000 
Accuracy = 0.904703 
Time Taken: 0.15 
 
Evaluated: 3134000 
Accuracy = 0.904711 
Time Taken: 0.16 
 
Evaluated: 3135000 
Accuracy = 0.904716 
Time Taken: 0.15 
 
Evaluated: 3136000 
Accuracy = 0.904715 
Time Taken: 0.15 
 
Evaluated: 3137000 
Accuracy = 0.904719 
Time Taken: 0.15 
 
Evaluated: 3138000 
Accu

Evaluated: 3257000 
Accuracy = 0.905239 
Time Taken: 0.15 
 
Evaluated: 3258000 
Accuracy = 0.905244 
Time Taken: 0.15 
 
Evaluated: 3259000 
Accuracy = 0.905250 
Time Taken: 0.15 
 
Evaluated: 3260000 
Accuracy = 0.905260 
Time Taken: 0.15 
 
Evaluated: 3261000 
Accuracy = 0.905263 
Time Taken: 0.15 
 
Evaluated: 3262000 
Accuracy = 0.905257 
Time Taken: 0.15 
 
Evaluated: 3263000 
Accuracy = 0.905230 
Time Taken: 0.15 
 
Evaluated: 3264000 
Accuracy = 0.905226 
Time Taken: 0.15 
 
Evaluated: 3265000 
Accuracy = 0.905221 
Time Taken: 0.15 
 
Evaluated: 3266000 
Accuracy = 0.905224 
Time Taken: 0.15 
 
Evaluated: 3267000 
Accuracy = 0.905227 
Time Taken: 0.15 
 
Evaluated: 3268000 
Accuracy = 0.905223 
Time Taken: 0.16 
 
Evaluated: 3269000 
Accuracy = 0.905227 
Time Taken: 0.15 
 
Evaluated: 3270000 
Accuracy = 0.905232 
Time Taken: 0.15 
 
Evaluated: 3271000 
Accuracy = 0.905232 
Time Taken: 0.15 
 
Evaluated: 3272000 
Accuracy = 0.905237 
Time Taken: 0.15 
 
Evaluated: 3273000 
Accu

Evaluated: 3392000 
Accuracy = 0.905346 
Time Taken: 0.15 
 
Evaluated: 3393000 
Accuracy = 0.905347 
Time Taken: 0.15 
 
Evaluated: 3394000 
Accuracy = 0.905346 
Time Taken: 0.15 
 
Evaluated: 3395000 
Accuracy = 0.905340 
Time Taken: 0.15 
 
Evaluated: 3396000 
Accuracy = 0.905337 
Time Taken: 0.16 
 
Evaluated: 3397000 
Accuracy = 0.905336 
Time Taken: 0.15 
 
Evaluated: 3398000 
Accuracy = 0.905334 
Time Taken: 0.15 
 
Evaluated: 3399000 
Accuracy = 0.905326 
Time Taken: 0.15 
 
Evaluated: 3400000 
Accuracy = 0.905327 
Time Taken: 0.15 
 
Evaluated: 3401000 
Accuracy = 0.905320 
Time Taken: 0.15 
 
Evaluated: 3402000 
Accuracy = 0.905322 
Time Taken: 0.15 
 
Evaluated: 3403000 
Accuracy = 0.905324 
Time Taken: 0.15 
 
Evaluated: 3404000 
Accuracy = 0.905329 
Time Taken: 0.15 
 
Evaluated: 3405000 
Accuracy = 0.905323 
Time Taken: 0.15 
 
Evaluated: 3406000 
Accuracy = 0.905320 
Time Taken: 0.15 
 
Evaluated: 3407000 
Accuracy = 0.905323 
Time Taken: 0.15 
 
Evaluated: 3408000 
Accu

Evaluated: 3527000 
Accuracy = 0.905560 
Time Taken: 0.15 
 
Evaluated: 3528000 
Accuracy = 0.905563 
Time Taken: 0.15 
 
Evaluated: 3529000 
Accuracy = 0.905564 
Time Taken: 0.15 
 
Evaluated: 3530000 
Accuracy = 0.905565 
Time Taken: 0.15 
 
Evaluated: 3531000 
Accuracy = 0.905566 
Time Taken: 0.15 
 
Evaluated: 3532000 
Accuracy = 0.905566 
Time Taken: 0.15 
 
Evaluated: 3533000 
Accuracy = 0.905569 
Time Taken: 0.15 
 
Evaluated: 3534000 
Accuracy = 0.905566 
Time Taken: 0.15 
 
Evaluated: 3535000 
Accuracy = 0.905573 
Time Taken: 0.15 
 
Evaluated: 3536000 
Accuracy = 0.905570 
Time Taken: 0.15 
 
Evaluated: 3537000 
Accuracy = 0.905576 
Time Taken: 0.15 
 
Evaluated: 3538000 
Accuracy = 0.905578 
Time Taken: 0.15 
 
Evaluated: 3539000 
Accuracy = 0.905582 
Time Taken: 0.15 
 
Evaluated: 3540000 
Accuracy = 0.905591 
Time Taken: 0.15 
 
Evaluated: 3541000 
Accuracy = 0.905596 
Time Taken: 0.15 
 
Evaluated: 3542000 
Accuracy = 0.905599 
Time Taken: 0.15 
 
Evaluated: 3543000 
Accu